In [1]:
import os
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

In [2]:
os.chdir("C:\\users\\mfell\\documents")

In [3]:
data = pd.read_csv('Samples_w_lithology.csv',header = None, names = ['Project','HoleID','depth','grade','Lith'])

c:\python27\arcgis10.2\lib\site-packages\IPython\core\interactiveshell.py:2705: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
import arcpy

In [5]:
arcpy.env.workspace = 'Database Connections\\KL_Exploration.sde'

In [11]:
domain = arcpy.da.ListDomains()[14]

In [12]:
domain.name

u'General_Geol'

In [13]:
data['Geol'] = data.Lith

In [14]:
import matplotlib.pyplot as plt


In [15]:
data['Geol'] = data.Geol.map(domain.codedValues)

In [16]:
data[data.grade.isnull()] = 0

In [17]:
data[data.grade.isnull()]

,Project,HoleID,depth,grade,Lith,Geol


In [18]:
data[data.grade.str.contains('t')==True] = 0
data[data.grade.str.contains('-') == True] = 0

In [19]:
data[data.grade.str.contains('-') == True]

,Project,HoleID,depth,grade,Lith,Geol


In [20]:
akdata = data[data.Project == 'Amalgamated Kirkland']

In [21]:
akdata['ID'] = akdata.index

c:\python27\arcgis10.2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
akpivot = akdata.pivot('ID', 'Geol','grade')

In [23]:
for x in akpivot.columns:
    akpivot[x] = akpivot[x].astype(np.float64)

In [20]:
akmeds = akpivot.median()
akmeds = akmeds.sort_values()
akpivot = akpivot.reindex(columns = akmeds.index)

In [21]:
akglist = [[x for x in akpivot[y] if not np.isnan(x)] for y in akpivot.columns]
akglist

[[0.02,
  0.01,
  0.01,
  0.02,
  0.0,
  0.01,
  0.02,
  0.0,
  0.0,
  0.01,
  0.01,
  0.0,
  0.0,
  0.0,
  0.01,
  0.01,
  0.0,
  0.0,
  0.02,
  0.0,
  0.01,
  0.0,
  0.0,
  0.0],
 [0.040000000000000001,
  0.050000000000000003,
  0.10000000000000001,
  0.0,
  0.01,
  0.0,
  0.070000000000000007,
  0.080000000000000002,
  0.01,
  0.01,
  0.0,
  0.0050000000000000001,
  0.0050000000000000001,
  0.01,
  0.0050000000000000001,
  0.029999999999999999,
  0.029999999999999999,
  0.059999999999999998,
  0.01,
  0.050000000000000003,
  0.01,
  0.0,
  0.0,
  0.02,
  0.080000000000000002,
  0.059999999999999998,
  0.040000000000000001,
  0.0,
  0.0,
  0.01,
  0.35999999999999999,
  0.02,
  0.01,
  0.01,
  0.02,
  0.0,
  0.0,
  0.0,
  0.02,
  0.01,
  0.029999999999999999,
  0.02,
  0.0,
  0.0,
  0.0,
  0.01,
  0.0025000000000000001,
  0.0025000000000000001,
  0.0025000000000000001,
  0.0025000000000000001,
  0.0025000000000000001,
  0.0025000000000000001,
  0.037999999999999999,
  0.0025000000000

In [24]:
aktest = akdata.groupby('Geol')


In [24]:
fig, axes = plt.subplots(nrows = 1, ncols = 1)
axes.boxplot(akglist )#, labels = pvt.columns)
axes.set_yscale('log')
axes.set_xlim(0.5, len(akpivot.columns)+ 0.5)
#axes.set_ylim((0, 100))
labels = []
for k, v in Series(akpivot.count()).iteritems():
    labels.append(str(k) +" - " + str(v))
xtickNames = plt.setp(axes, xticklabels=labels )
plt.setp(xtickNames, rotation = 70)
axes.set_title('Amalgamated\nKirkland')
plt.show()

In [ ]:
akpivot = akpivot.reindex(columns = akmeds.index)


In [ ]:
this = Series(akpivot.count())
this

In [27]:
plt.hist(akdata[akdata.Geol == 'Sediments'].grade.astype(np.float64), bins =50) #np.logspace(0.1, 2, 50), )
plt.title('Amalgamated Kirkland\nAssay Frequency in Sediments')
#plt.xscale('log')
plt.yscale('log')#, nonposy='clip')
plt.show()
